<a href="https://colab.research.google.com/github/adem0204/Chatbot-juridique/blob/main/code_projet_Chatbot_juridique_avec_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers faiss-cpu PyPDF2 gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.1 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Étape 1 : Charger et extraire le texte des PDFs

In [5]:
from PyPDF2 import PdfReader

def extract_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# Exemple : mettre le chemin vers ton PDF
text = extract_text("/content/drive/MyDrive/projet nlp/Code du travail - Légifrance.pdf")
print(text[:500])  # affiche le début pour vérifier


Co de d u travail
 Dernière mise à jour des données de ce code : 10 décembre 2025
 Télécharger le code à la date du : 16 déc. 2025
V ersion en vigueur a u 16 décembr e 2025
P ar tie législativ e (Ar ticles L1 à L8331-1)
Chapitr e pr éliminair e : Dial ogue social. (Ar ticles L1 à L3)
Article L1 Article L2 Article L3 
Pr emièr e par tie : Les r ela tions in dividuelles de tr a v ail (Ar ticles L1111-1 à L1532-1)
Livr e Ier : Dispositions pr éliminair es (Ar ticles L1111-1 à L1155-2)
Titr e Ier 


Étape 2 : Découper le texte en chunks

In [6]:
def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

chunks = chunk_text(text)
print(f"Nombre de chunks : {len(chunks)}")
print(chunks[0])  # vérifier le premier chunk


Nombre de chunks : 479
Co de d u travail  Dernière mise à jour des données de ce code : 10 décembre 2025  Télécharger le code à la date du : 16 déc. 2025 V ersion en vigueur a u 16 décembr e 2025 P ar tie législativ e (Ar ticles L1 à L8331-1) Chapitr e pr éliminair e : Dial ogue social. (Ar ticles L1 à L3) Article L1 Article L2 Article L3 Pr emièr e par tie : Les r ela tions in dividuelles de tr a v ail (Ar ticles L1111-1 à L1532-1) Livr e Ier : Dispositions pr éliminair es (Ar ticles L1111-1 à L1155-2) Titr e Ier : Champ d' app lica tion et calcul des seuils d' eff ectifs (Ar ticles L1111-1 à L1111- 3) Chapitr e unique. (Ar ticles L1111-1 à L1111-3) Article L1111-1 Article L1111-2 Article L1111-3 Titr e II : Dr oits et liber tés dans l' entr eprise (Ar ticles L1121-1 à L1121-2) Chapitr e unique. (Ar ticles L1121-1 à L1121-2) Article L1121-1 Article L1121-2 Titr e III : Discrimina tions (Ar ticles L1131-1 à L1134-5) Chapitr e Ier : Champ d' app lica tion. (Ar ticles L1131-1 à L1131-2

ÉTAPE 3 : Embeddings + Index FAISS (le cœur du RAG)

In [7]:
!pip install sentence-transformers faiss-cpu


3.2 Charger le modèle d’embeddings

On utilise Sentence-BERT (très bon pour le RAG).


In [8]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

3.3 Calculer les embeddings des chunks

In [9]:
import numpy as np

embeddings = embedding_model.encode(
    chunks,
    show_progress_bar=True,
    convert_to_numpy=True
)

print("Shape des embeddings :", embeddings.shape)


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Shape des embeddings : (479, 384)


3.4 Créer l’index FAISS


In [10]:
import faiss

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("Nombre de vecteurs dans l'index :", index.ntotal)


Nombre de vecteurs dans l'index : 479


3.5 Test du retrieval (super important)

In [12]:
def retrieve_chunks(question, k=3):
    question_embedding = embedding_model.encode([question])
    distances, indices = index.search(question_embedding, k)
    return [chunks[i] for i in indices[0]]

question = "Quels sont les droits du salarié en cas de licenciement ?"
results = retrieve_chunks(question)

for i, res in enumerate(results):
    print(f"\n--- Passage {i+1} ---\n")
    print(res[:500])



--- Passage 1 ---

es a u dr oit d' expr ession des salariés. (a br ogé) Chapitr e II : Dispositions comp lém entair es r ela tiv es a u dr oit d' expr ession des salariés dans les entr eprises et éta b lissem ents du secteur pub lic. (a br ogé) Titr e VI : DISPOSITIONS COMMUNES RELA TIVES A U DROIT D'EXPRESSION DES SALARIES (a br ogé) Chapitr e Ier : DISPOSITIONS COMMUNES RELA TIVES A U DROIT D'EXPRESSION DES SALARIES. (a br ogé) Titr e VII : PEN ALITES (a br ogé) Chapitr e Ier : LES SYNDICA T S PROFESSIONNELS (a

--- Passage 2 ---

m embr e d'un e commission paritair e d'hy gièn e, de sécurité et des con ditions de tr a v ail en agricultur e. (Ar ticle L2412-9) Article L2412-9 Section 10 : Salarié man da té. (Ar ticle L2412-10) Article L2412-10 Section 11 : Membr e du conseil ou administr a teur d'un e caisse de sécurité sociale. (Ar ticle L2412-11) Article L2412-11 Section 12 : Repr ésentant des salariés dans un e chambr e d' agricultur e. (Ar ticle L2412-12) Article L2412-12 Secti

ÉTAPE 4 : Génération de réponse avec LLM (RAG complet)

4.1 Installer le LLM


In [13]:
!pip install transformers accelerate


4.2 Charger le modèle LLM

On prend un modèle léger mais efficace pour QA.


In [14]:
from transformers import pipeline

llm = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    max_new_tokens=300,
    temperature=0.2
)


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cpu


4.3 Construire le prompt RAG

In [17]:
def build_prompt(question, retrieved_chunks):
    context = "\n\n".join(retrieved_chunks)

    prompt = f"""
Tu es un assistant juridique spécialisé en droit du travail français.

Réponds uniquement à partir des extraits suivants.
Si la réponse n'est pas présente, dis clairement : "Je ne sais pas".

Extraits :
{context}

Question :
{question}

Réponse (avec citation des articles ou sources si possible) :
"""
    return prompt


4.4 Générer la réponse finale

In [18]:
prompt = build_prompt(question, results)

response = llm(prompt)[0]["generated_text"]

print(response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Tu es un assistant juridique spécialisé en droit du travail français.

Réponds uniquement à partir des extraits suivants.
Si la réponse n'est pas présente, dis clairement : "Je ne sais pas".

Extraits :
es a u dr oit d' expr ession des salariés. (a br ogé) Chapitr e II : Dispositions comp lém entair es r ela tiv es a u dr oit d' expr ession des salariés dans les entr eprises et éta b lissem ents du secteur pub lic. (a br ogé) Titr e VI : DISPOSITIONS COMMUNES RELA TIVES A U DROIT D'EXPRESSION DES SALARIES (a br ogé) Chapitr e Ier : DISPOSITIONS COMMUNES RELA TIVES A U DROIT D'EXPRESSION DES SALARIES. (a br ogé) Titr e VII : PEN ALITES (a br ogé) Chapitr e Ier : LES SYNDICA T S PROFESSIONNELS (a br ogé) SEC TION 1 : ST A TUT JURIDIQUE DES SYNDICA T S. (a br ogé) SEC TION 2 : EXERCICE DU DROIT SYNDICAL D ANS LES ENTREPRISES E T MARQUES SYNDICALES. (a br ogé) Chapitr e II : LES DELEGUES DU PERSONNEL. (a br ogé) Chapitr e III : LES COMITES D'ENTREPRISE (a br ogé) Titr e VII : F ONDS SALAR

ÉTAPE 5 : Interface Chatbot avec Gradio

5.1 Installer Gradio

In [5]:
!pip install gradio transformers accelerate



5.2 Chargement du LLM

In [12]:
from transformers import pipeline

llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=300
)


Device set to use cpu


5.3 Prompt RAG

In [13]:
def build_prompt(question, retrieved_chunks):
    context = "\n\n".join(retrieved_chunks)
    return f"""
You are a legal assistant specialized ONLY in the French Labour Code (Code du travail).

Answer the question ONLY using the articles from the Code du travail provided below.
If the answer is not found in these articles, say exactly: Je ne sais pas.

Code du travail excerpts:
{context}

Question:
{question}

Answer (mention articles if possible):
"""


5.4 Fonction Chatbot

In [14]:
def chatbot(question):
    try:
        retrieved = retrieve_chunks(question, k=3)
        prompt = build_prompt(question, retrieved)
        result = llm(prompt)
        return result[0]["generated_text"]
    except Exception as e:
        return f"Erreur interne : {str(e)}"


5.5 Interface Gradio

In [ ]:
import gradio as gr

interface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Posez votre question en droit du travail..."),
    outputs=gr.Textbox(label="Réponse du chatbot"),
    title="🤖 Chatbot juridique – Droit du travail (RAG)",
    description="Ce chatbot répond à des questions juridiques simples à partir du Code du travail."
)

interface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7f8d6fcfd60b67498b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
